In [3]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import time
import random
import requests
client_id = "6f8bb3e64c2944d586037b9907043601"
client_secret = "81f5f44660364831be8d9118ccb51785"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [40]:
singers = pd.read_csv('singers.csv')
singers

,Singers,Genre
0,Nusrat Fateh Ali Khan,Qawwali
1,Fateh Ali Khan,Qawwali
2,Sabri Brothers,Qawwali
3,Mohammad Rafi,Qawwali
4,Aziz Mian,Qawwali
...,...,...
85,Lil Baby,Hip-Hop
86,Lil Wayne,Hip-Hop
87,DJ Khaled,Hip-Hop
88,Pitbull,Hip-Hop


In [42]:
for i in range(len(singers)):
    artist = singers.loc[i,'Singers']
    result = sp.search(artist)
    singers.loc[i, 'URI'] = result['tracks']['items'][0]['artists'][0]['uri']

In [43]:
singers.to_csv('singersgenreuri.csv')

In [51]:
artist_uri = []
track_uri = []
track_preview = []
for artist in singers['URI']:
    results = sp.artist_top_tracks(artist)
    for track in results['tracks'][:10]:
        try:  
            artist_uri.append(artist)
            track_uri.append(track['uri'])
            track_preview.append(track['preview_url'])
        except:
            next    

In [52]:
track_df = pd.DataFrame()
track_df['Artist'] = artist_uri
track_df['TrackURI'] = track_uri
track_df['Preview'] = track_preview

In [53]:
track_df.to_csv('track_df.csv')

In [54]:
total_df = pd.merge(singers, track_df, how = 'outer', left_on='URI', right_on = 'Artist')
total_df.to_csv("singers, genre, uri, artist, trackuri.csv")

In [17]:
total_df

NameError: name 'total_df' is not defined

In [71]:
total_df2 = total_df.dropna()

In [76]:
total_df2

,Singers,Genre,Artist,TrackURI,Preview
0,Nusrat Fateh Ali Khan,Qawwali,spotify:artist:5HcunTidTUrOaf8V0iJcvl,spotify:track:2ufLEVbzvEcJ3subW8jBUp,https://p.scdn.co/mp3-preview/60e40221b91d510b...
1,Nusrat Fateh Ali Khan,Qawwali,spotify:artist:5HcunTidTUrOaf8V0iJcvl,spotify:track:4sNiMTu47WkUgW87qeUlWF,https://p.scdn.co/mp3-preview/5e7533916e316d37...
2,Nusrat Fateh Ali Khan,Qawwali,spotify:artist:5HcunTidTUrOaf8V0iJcvl,spotify:track:4tMwHkxBwnGkts9mxFpwQy,https://p.scdn.co/mp3-preview/c14827a65208f44b...
3,Nusrat Fateh Ali Khan,Qawwali,spotify:artist:5HcunTidTUrOaf8V0iJcvl,spotify:track:5wNpnKRUxbPoIpDXFZ5NkU,https://p.scdn.co/mp3-preview/66aaf3d7dad3d076...
4,Nusrat Fateh Ali Khan,Qawwali,spotify:artist:5HcunTidTUrOaf8V0iJcvl,spotify:track:46Y6dCrKdAiQeelp9px3ww,https://p.scdn.co/mp3-preview/2158f8af0250e98d...
...,...,...,...,...,...
959,Pitbull,Hip-Hop,spotify:artist:0TnOYISbd1XYRBk9myaseg,spotify:track:3rfhI32Il2hVRKDkuGeeen,https://p.scdn.co/mp3-preview/bccf42f49e65f845...
960,Pitbull,Hip-Hop,spotify:artist:0TnOYISbd1XYRBk9myaseg,spotify:track:1PCvKFPWnTXAe2oaReVUcr,https://p.scdn.co/mp3-preview/faa5543826cbcfc6...
961,Ludacris,Hip-Hop,spotify:artist:1uNFoZAHBGtllmzznpCI3s,spotify:track:5PjdY0CKGZdEuoNab3yDmX,https://p.scdn.co/mp3-preview/40a8daf1f3cbb5c6...
969,Ludacris,Hip-Hop,spotify:artist:1uNFoZAHBGtllmzznpCI3s,spotify:track:4j5ffIFh7bFT7GZciP1TCy,https://p.scdn.co/mp3-preview/35dde02aaaf1711d...


In [77]:
df = pd.DataFrame(total_df2['TrackURI'])
df.columns = ['TrackURI']

for i in range(0,df.shape[0]):
  try:
    time.sleep(random.uniform(3, 6))
    URI = df.TrackURI[i]
    features = sp.audio_features(URI)
    track = sp.track(URI)
    df.loc[i, 'track'] = track['name']
    df.loc[i,'acousticness'] = features[0]['acousticness']
    df.loc[i,'instrumentalness'] = features[0]['instrumentalness']
    df.loc[i,'energy'] = features[0]['energy']
    df.loc[i,'speechiness'] = features[0]['speechiness']
    df.loc[i,'liveness'] = features[0]['liveness']
    df.loc[i,'loudness'] = features[0]['loudness']
    df.loc[i,'danceability'] = features[0]['danceability']
    df.loc[i,'tempo'] = features[0]['tempo']
    df.loc[i,'valence'] = features[0]['valence']
    uri=0
  except:
    next

In [82]:
df.to_csv('full_spotify.csv')

In [87]:
new_df = df[-df['track'].isna()]

In [93]:
new_df2 = pd.merge(total_df2, new_df, how = 'left', left_on = 'TrackURI', right_on = 'TrackURI')

In [99]:
new_df2 = new_df2[-new_df2['track'].isna()]

In [102]:
new_df2.to_csv('spotify_clean.csv')

In [4]:
new_df2 = pd.read_csv('../spotify_clean.csv')

In [5]:
new_df2.reset_index(inplace = True)

In [16]:
new_df2['Genre'].value_counts()

Metal      99
Qawwali    89
Ghazal     87
Rock       78
Pop        71
Blues      55
Hip-Hop    11
Jazz        1
Name: Genre, dtype: int64

In [15]:
for i in range(len(new_df2)):
    try:
        r = requests.get(new_df2.loc[i, 'Preview'])
        open('{}_{}.wav'.format(new_df2.loc[i,'Genre'], i), 'wb').write(r.content)
    except:
        next